# Lesson 7: Build a Live Camera App

Rita Philavanh

20170905

## Retrieve Street View House Number (SVHN) training/test dataset

Download train.tar.gz and test.tar.gz files from http://ufldl.stanford.edu/housenumbers/ 

## Pre-processing

### Extract dataset from compressed tar.gz file

In [1]:
import os
import sys
import tarfile
import numpy as np

import time

In [2]:

num_classes = 0
np.random.seed(133)
data_root = '.'

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract('train.tar.gz')
test_folders = maybe_extract('test.tar.gz')

train already present - Skipping extraction of train.tar.gz.


KeyboardInterrupt: 

### Extract labels from .mat file 

(used digiStruct.py code found here: https://github.com/prijip/Py-Gsvhn-DigitStruct-Reader/blob/360a03909a22a28e2e814e83d6915ee5f915e5da/digitStruct.py)



In [2]:
#import h5py
#import numpy as np
#digitStruct = h5py.File('train/digitStruct.mat')
#digitStruct.keys()
#mat = {}
#for k, v in digitStruct.items():
#    print(k,v)
#    mat[k] = np.array(v)

#print(mat)


import h5py
import numpy as np

#
# Bounding Box
#
class BBox:
    def __init__(self):
        self.label = ""     # Digit
        self.left = 0
        self.top = 0
        self.width = 0
        self.height = 0

class DigitStruct:
    def __init__(self):
        self.name = None    # Image file name
        self.bboxList = None # List of BBox structs

# Function for debugging
def printHDFObj(theObj, theObjName):
    isFile = isinstance(theObj, h5py.File)
    isGroup = isinstance(theObj, h5py.Group)
    isDataSet = isinstance(theObj, h5py.Dataset)
    isReference = isinstance(theObj, h5py.Reference)
    print("{}".format(theObjName))
    print("    type(): {}".format(type(theObj)))
    if isFile or isGroup or isDataSet:
        # if theObj.name != None:
        #    print "    name: {}".format(theObj.name)
        print("    id: {}".format(theObj.id))
    if isFile or isGroup:
        print("    keys: {}".format(theObj.keys()))
    if not isReference:
        print("    Len: {}".format(len(theObj)))

    if not (isFile or isGroup or isDataSet or isReference):
        print(theObj)

def readDigitStructGroup(dsFile):
    dsGroup = dsFile["digitStruct"]
    return dsGroup

#
# Reads a string from the file using its reference
#
def readString(strRef, dsFile):
    strObj = dsFile[strRef]
    str = ''.join(chr(i) for i in strObj)
    return str

#
# Reads an integer value from the file
#
def readInt(intArray, dsFile):
    intRef = intArray[0]
    isReference = isinstance(intRef, h5py.Reference)
    intVal = 0
    if isReference:
        intObj = dsFile[intRef]
        intVal = int(intObj[0])
    else: # Assuming value type
        intVal = int(intRef)
    return intVal

def yieldNextInt(intDataset, dsFile):
    for intData in intDataset:
        intVal = readInt(intData, dsFile)
        yield intVal 

def yieldNextBBox(bboxDataset, dsFile):
    for bboxArray in bboxDataset:
        bboxGroupRef = bboxArray[0]
        bboxGroup = dsFile[bboxGroupRef]
        labelDataset = bboxGroup["label"]
        leftDataset = bboxGroup["left"]
        topDataset = bboxGroup["top"]
        widthDataset = bboxGroup["width"]
        heightDataset = bboxGroup["height"]

        left = yieldNextInt(leftDataset, dsFile)
        top = yieldNextInt(topDataset, dsFile)
        width = yieldNextInt(widthDataset, dsFile)
        height = yieldNextInt(heightDataset, dsFile)

        bboxList = []

        for label in yieldNextInt(labelDataset, dsFile):
            bbox = BBox()
            bbox.label = label
            bbox.left = next(left)
            bbox.top = next(top)
            bbox.width = next(width)
            bbox.height = next(height)
            bboxList.append(bbox)

        yield bboxList

def yieldNextFileName(nameDataset, dsFile):
    for nameArray in nameDataset:
        nameRef = nameArray[0]
        name = readString(nameRef, dsFile)
        yield name

# dsFile = h5py.File('train/digitStruct.mat', 'r')
def yieldNextDigitStruct(dsFileName):
    dsFile = h5py.File(dsFileName, 'r')
    dsGroup = readDigitStructGroup(dsFile)
    nameDataset = dsGroup["name"]
    bboxDataset = dsGroup["bbox"]

    bboxListIter = yieldNextBBox(bboxDataset, dsFile)
    for name in yieldNextFileName(nameDataset, dsFile):
        bboxList = next(bboxListIter)
        obj = DigitStruct()
        obj.name = name
        obj.bboxList = bboxList
        yield obj

L = []
y1 = []
y2 = []
y3 = []
y4 = []
y5 = []
def testMain():
    i = 0
    dsFileName = 'train/digitStruct.mat'
    testCounter = 0

    for dsObj in yieldNextDigitStruct(dsFileName):
        # testCounter += 1
       # print(dsObj.name)
      #  print(len(dsObj.bboxList))
        L.append(len(dsObj.bboxList))
        
        cnty = 0
        for bbox in dsObj.bboxList:
         #   print("    {}:{},{},{},{}".format(
         #       bbox.label, bbox.left, bbox.top, bbox.width, bbox.height))
            
            if cnty == 0: y1.append(bbox.label)
            elif cnty == 1: y2.append(bbox.label) 
            elif cnty == 2: y3.append(bbox.label)
            elif cnty == 3: y4.append(bbox.label)
            elif cnty == 4: y5.append(bbox.label)
            
            cnty += 1
            
        if cnty < 5:
            if len(dsObj.bboxList) < 1: y1.append(10)
            if len(dsObj.bboxList) < 2: y2.append(10) 
            if len(dsObj.bboxList) < 3: y3.append(10)
            if len(dsObj.bboxList) < 4: y4.append(10)
            if len(dsObj.bboxList) < 5: y5.append(10)
        if testCounter >= 5:
            break
        i +=1
        
if __name__ == "__main__":
    t1 = time.time()
    testMain()
    t2 = time.time()
    print("Time: %0.2fs" % (t2 - t1))

Time: 262.64s


### Check Labels

In [3]:
print(len(L))
print(L[:10]) # length of digits
print(y1[:10]) # 1st digit
print(y2[:10]) # 2nd digit
print(y3[:10]) # ...etc (If no digit, then 'None')
print(y4[:10])
print(y5[:10])

33402
[2, 2, 2, 2, 2, 2, 2, 3, 3, 2]
[1, 2, 2, 9, 3, 3, 2, 7, 1, 1]
[9, 3, 5, 3, 1, 3, 8, 4, 2, 6]
[10, 10, 10, 10, 10, 10, 10, 4, 8, 10]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


### Convert dataset to 32x32 pixels grayscale matrix

In [4]:
import cv2

t1 = time.time()

X = []
pic_num = 1
#print(len(os.listdir('./train')))
for i in range(len(os.listdir('./train'))): 
    img = cv2.imread("./train/"+str(pic_num) +".png",cv2.IMREAD_GRAYSCALE)
    resized_image = cv2.resize(img, (32, 32))
    X.append(resized_image)
    pic_num +=1

t2 = time.time()
print("Time: %0.2fs" % (t2 - t1))

error: C:\projects\opencv-python\opencv\modules\imgproc\src\imgwarp.cpp:3483: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize


### Split dataset into train and validation set

In [5]:
X_valid = X[:10000]
X_train = X[10000:]

print(len(X), len(X_valid), len(X_train))

L_valid = L[:10000] 
L_train = L[10000:] 
y1_valid = y1[:10000]
y2_valid = y2[:10000]
y3_valid = y3[:10000]
y4_valid = y4[:10000]
y5_valid = y5[:10000]

y1_train = y1[10000:]
y2_train = y2[10000:]
y3_train = y3[10000:]
y4_train = y4[10000:]
y5_train = y5[10000:]

print(len(L), len(L_valid), len(L_train))

33402 10000 23402
33402 10000 23402


In [6]:
image_size = 32 # 32 x 32 pixel
num_channels = 1 # grayscale
num_labels = 11 # 0-9 + 10 (for missing value)
num_digits = 6 #max 5 digit for housenumber

import numpy as np

def reformat(dataset, label0, label1, label2, label3, label4, label5):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
  print((label0[:10]))
  print((label1[:10]))
  print((label2[:10]))
  print((label3[:10]))
  print(np.asarray(label3[:10]))
  label0 = (np.arange(num_digits) == np.asarray(label0)[:,None]).astype(np.float32)
  label1 = (np.arange(num_labels) == np.asarray(label1)[:,None]).astype(np.float32)
  label2 = (np.arange(num_labels) == np.asarray(label2)[:,None]).astype(np.float32)
  label3 = (np.arange(num_labels) == np.asarray(label3)[:,None]).astype(np.float32)
  label4 = (np.arange(num_labels) == np.asarray(label4)[:,None]).astype(np.float32)
  label5 = (np.arange(num_labels) == np.asarray(label5)[:,None]).astype(np.float32)

  print(np.asarray(label3[:10]))
  return dataset, label0, label1, label2, label3, label4, label5
train_dataset, train_label0, train_label1, train_label2, train_label3, train_label4, train_label5 = reformat(np.asarray(X_train), L_train, y1_train, y2_train, y3_train, y4_train, y5_train)
valid_dataset, valid_label0, valid_label1, valid_label2, valid_label3, valid_label4, valid_label5 = reformat(np.asarray(X_valid), L_valid, y1_valid, y2_valid, y3_valid, y4_valid, y5_valid)
#test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_label0.shape, train_label1.shape)
print('Validation set', valid_dataset.shape, valid_label0.shape, valid_label1.shape)
#print('Test set', test_dataset.shape, test_labels.shape)

[2, 3, 2, 2, 4, 2, 2, 2, 1, 2]
[4, 2, 5, 8, 1, 4, 1, 9, 6, 1]
[1, 1, 9, 8, 10, 2, 6, 2, 10, 3]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[10 10 10 10 10 10 10 10 10 10]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]
[2, 2, 2, 2, 2, 2, 2, 3, 3, 2]
[1, 2, 2, 9, 3, 3, 2, 7, 1, 1]
[9, 3, 5, 3, 1, 3, 8, 4, 2, 6]
[10, 10, 10, 10, 10, 10, 10, 4, 8, 10]
[10 10 10 10 10 10 10  4  8 10]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.

# Import Modules 

In [7]:
import tensorflow as tf
import numpy as np

# Neural Network Configurations

In [8]:
batch_size = 256

# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

#Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36        # There are 36 of these filters.

#Convolutional Layer 2.
filter_size3 = 5          # Convolution filters are 5 x 5 pixels.
num_filters3 = 64        # There are 64 of these filters.

# Fully-connected layer.
fc_size = 1024         # Number of neurons in fully-connected hidden layer

drop_rate = 0.9

# Create graph

### Declare variables to initialize

In [9]:
graph = tf.Graph()

with graph.as_default():
#with tf.device('/gpu:0'):
  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels0 = tf.placeholder(tf.float32, shape=(batch_size, num_digits))
  tf_train_labels1 = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_train_labels2 = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_train_labels3 = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_train_labels4 = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_train_labels5 = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset =  tf.constant(valid_dataset[:batch_size]) #tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels)) #
 # tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  #w1 = tf.get_variable("w1", shape=[filter_size1, filter_size1, num_channels, num_filters1], initializer=tf.contrib.layers.xavier_initializer()) 
  #w1 = tf.Variable(tf.truncated_normal([filter_size1, filter_size1, num_channels, num_filters1], stddev=0.1)) # 5, 5, 1, 16
  w1cv1 = tf.Variable(tf.truncated_normal([filter_size1,filter_size1, num_channels, num_filters1], stddev=np.sqrt(1/(filter_size1*filter_size1*num_channels)))) # 5, 5, 1, 16
  b1cv1 = tf.Variable(tf.zeros([num_filters1])) #16

  w1cv2 = tf.Variable(tf.truncated_normal([filter_size1,filter_size1, num_filters1, num_filters1], stddev=np.sqrt(1/(filter_size1*filter_size1*num_filters1)))) # 5, 5, 16, 16
  b1cv2 = tf.Variable(tf.constant(0.1, shape=[num_filters1])) #16
    
  w1cv3 = tf.Variable(tf.truncated_normal([filter_size1,filter_size1, num_filters1, num_filters1], stddev=np.sqrt(1/(filter_size1*filter_size1*num_filters1)))) # 5, 5, 16, 16
  b1cv3 = tf.Variable(tf.constant(0.1, shape=[num_filters1])) #16

#pool 16

  w2cv1 = tf.Variable(tf.truncated_normal([filter_size2,filter_size2, num_filters1, num_filters2], stddev=np.sqrt(1/(filter_size1*filter_size2*num_filters1)))) # 5, 5, 16, 36
  b2cv1 = tf.Variable(tf.constant(0.1, shape=[num_filters2])) #36

  w2cv2 = tf.Variable(tf.truncated_normal([filter_size2,filter_size2, num_filters2, num_filters2], stddev=np.sqrt(1/(filter_size1*filter_size2*num_filters2)))) # 5, 5, 36, 36
  b2cv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2])) #36

  w2cv3 = tf.Variable(tf.truncated_normal([filter_size2,filter_size2, num_filters2, num_filters2], stddev=np.sqrt(1/(filter_size1*filter_size2*num_filters2)))) # 5, 5, 36, 36
  b2cv3 = tf.Variable(tf.constant(0.1, shape=[num_filters2])) #36
  
#pool 16

  w3cv1 = tf.Variable(tf.truncated_normal([filter_size3,filter_size3, num_filters2, num_filters3], stddev=np.sqrt(1/(filter_size3*filter_size3*num_filters2)))) # 5, 5, 16, 64
  b3cv1 = tf.Variable(tf.constant(0.1, shape=[num_filters3])) #64

  w3cv2 = tf.Variable(tf.truncated_normal([filter_size3,filter_size3, num_filters3, num_filters3], stddev=np.sqrt(1/(filter_size3*filter_size3*num_filters3)))) # 5, 5, 64, 64
  b3cv2 = tf.Variable(tf.constant(0.1, shape=[num_filters3])) #64
  
  w3cv3 = tf.Variable(tf.truncated_normal([filter_size3,filter_size3, num_filters3, num_filters3], stddev=np.sqrt(1/(filter_size3*filter_size3*num_filters3)))) # 5, 5, 64, 64
  b3cv3 = tf.Variable(tf.constant(0.1, shape=[num_filters3])) #64
    
 #pool 8

  #wfcin = image_size // 2*2*2 * image_size // 2*2*2 *num_filters3 #(3 max pool @ stride 2)
  wfc = tf.Variable(tf.truncated_normal([fc_size, batch_size], stddev=np.sqrt(2/(fc_size + batch_size)))) #4*4*64=1024,256 
  bfc = tf.Variable(tf.constant(0.1, shape=[batch_size])) #256
    
  w0fc = tf.Variable(tf.truncated_normal([batch_size, num_digits], stddev=np.sqrt(2/(batch_size + num_digits)))) #256, 6
  b0fc = tf.Variable(tf.constant(0.1, shape=[num_digits])) #6
    
  w1fc = tf.Variable(tf.truncated_normal([batch_size, num_labels], stddev=np.sqrt(2/(batch_size + num_labels)))) #256, 11
  b1fc = tf.Variable(tf.constant(0.1, shape=[num_labels])) #11
  w2fc = tf.Variable(tf.truncated_normal([batch_size, num_labels], stddev=np.sqrt(2/(batch_size + num_labels)))) #256, 11
  b2fc = tf.Variable(tf.constant(0.1, shape=[num_labels])) #11
  w3fc = tf.Variable(tf.truncated_normal([batch_size, num_labels], stddev=np.sqrt(2/(batch_size + num_labels)))) #256, 11
  b3fc = tf.Variable(tf.constant(0.1, shape=[num_labels])) #11
  w4fc = tf.Variable(tf.truncated_normal([batch_size, num_labels], stddev=np.sqrt(2/(batch_size + num_labels)))) #256, 11
  b4fc = tf.Variable(tf.constant(0.1, shape=[num_labels])) #11
  w5fc = tf.Variable(tf.truncated_normal([batch_size, num_labels], stddev=np.sqrt(2/(batch_size + num_labels)))) #256, 11
  b5fc = tf.Variable(tf.constant(0.1, shape=[num_labels])) #11

#init = tf.contrib.layers.xaviar_initializer()
#h = tf.layers.dense(inputs=x, units=image_size, activation=tf.nn.relu, kernel_initializer=init)
#y_pred = tf.layers.dense(inputs=h, units = 1000, kernel_initializer=init)

# Model.
  def model(data):
    ######
    # CNN
    ######
    # x = 32x32 pixel * 1 channel* 256 batch samples
    # w = 5x5 filter * 1 channel * 16 filters/depth
    # b1= 16 depth
    # conv/hidden = 32x32 pixel * 16 depth * 256 batch samples  (32-5+(2*2)[padding to be same output])/1[stride]+1
    conv1a = tf.nn.conv2d(data, w1cv1, [1, 1, 1, 1], padding='SAME')
    hidden1a =tf.maximum((conv1a + b1cv1), 0.01 * (conv1a + b1cv1)) # tf.nn.relu(conv1 + b1) #
    
    # hidden = 32x32 pixel * 16 channel* 256 batch samples
    # w = 5x5 filter * 16 channel * 16 filters/depth
    # b = 16 depth
    # conv/hidden = 32x32 pixel * 16 depth * 256 batch samples  (32-5+(2*2)[padding to be same output])/1[stride]+1
    conv2a = tf.nn.conv2d(hidden1a, w1cv2, [1, 1, 1, 1], padding='SAME')
    hidden2a =tf.maximum((conv2a + b1cv2), 0.01 * (conv2a + b1cv2)) #tf.nn.relu(conv2 + b2)# 
    
    conv3a = tf.nn.conv2d(hidden2a, w1cv3, [1, 1, 1, 1], padding='SAME')
    hidden3a =tf.maximum((conv3a + b1cv3), 0.01 * (conv3a + b1cv3)) #tf.nn.relu(conv3 + b3)# 
    
    # pool = 16x16 pixel * 16 depth * 256 batch samples
    pool1a = tf.nn.max_pool(hidden3a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    ###
    
    # pool = 8x8 pixel * 36 channel* 256 batch samples
    # w = 5x5 filter * 36 channel * 36 filters/depth
    # b = 36 depth
    # conv/hidden = 8x8 pixel * 36 depth * 256 batch samples  (8-5+(2*2)[padding to be same output])/1[stride]+1
    conv1b = tf.nn.conv2d(pool1a, w2cv1, [1, 1, 1, 1], padding='SAME')
    hidden1b = tf.maximum((conv1b + b2cv1), 0.01 * (conv1b + b2cv1)) #tf.nn.relu(conv1b + b1b)#
    
    # hidden = 8x8 pixel * 36 depth * 256 batch samples 
    # w = 5x5 filter * 36 channel * 36 filters/depth
    # b = 64 depth
    # conv/hidden = 8x8 pixel * 64 depth * 256 batch samples  (8-5+(2*2)[padding to be same output])/1[stride]+1
    conv2b = tf.nn.conv2d(hidden1b, w2cv2, [1, 1, 1, 1], padding='SAME')
    hidden2b =tf.maximum((conv2b + b2cv2), 0.01 * (conv2b + b2cv2)) #tf.nn.relu(conv2b + b2b)# 
    #print(w.get_shape())
    #print(b.get_shape())
    #print(hidden.get_shape())
    
    conv3b = tf.nn.conv2d(hidden2b, w2cv3, [1, 1, 1, 1], padding='SAME')
    hidden3b =tf.maximum((conv3b + b2cv3), 0.01 * (conv3b + b2cv3)) #tf.nn.relu(conv3 + b3)# 
    
    # pool = 4x4 pixel * 36 depth * 256 batch samples
    pool1b = tf.nn.max_pool(hidden3b, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #print(pool.get_shape())
    
    ###
    
    # pool = 4x4 pixel * 36 channel* 256 batch samples
    # w = 5x5 filter * 36 channel * 64 filters/depth
    # b = 64 depth
    # conv/hidden = 4x4 pixel * 64 depth * 256 batch samples  (4-5+(2*2)[padding to be same output])/1[stride]+1
    conv1c = tf.nn.conv2d(pool1b, w3cv1, [1, 1, 1, 1], padding='SAME')
    hidden1c = tf.maximum((conv1c + b3cv1), 0.01 * (conv1c + b3cv1)) #tf.nn.relu(conv1b + b1b)#
    
    # hidden = 4x4 pixel * 64 depth * 256 batch samples 
    # w = 5x5 filter * 64 channel * 64 filters/depth
    # b = 64 depth
    # conv/hidden = 4x4 pixel * 64 depth * 256 batch samples  (4-5+(2*2)[padding to be same output])/1[stride]+1
    conv2c = tf.nn.conv2d(hidden1c, w3cv2, [1, 1, 1, 1], padding='SAME')
    hidden2c =tf.maximum((conv2c + b3cv2), 0.01 * (conv2c + b3cv2)) #tf.nn.relu(conv2b + b2b)# 
    #print(w.get_shape())
    #print(b.get_shape())
    #print(hidden.get_shape())
    
    conv3c = tf.nn.conv2d(hidden2c, w3cv3, [1, 1, 1, 1], padding='SAME')
    hidden3c =tf.maximum((conv3c + b3cv3), 0.01 * (conv3c + b3cv3)) #tf.nn.relu(conv3 + b3)# 
    
    # pool = 8x8 pixel * 64 depth * 256 batch samples
    pool1c = tf.nn.max_pool(hidden3c, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #print(pool.get_shape())
        
    #####
    # FC Layers
    #####
    shape = pool1c.get_shape().as_list()
   # print(shape)
    reshape = tf.reshape(pool1c, [shape[0], shape[1] * shape[2] * shape[3]]) #[256, 4, 4, 64]
   # print(reshape) #256 * 4*4*64=1024
   # print(wfc.get_shape())
   # print(bfc.get_shape())
    # w = 1024 depth * 1024 FC hidden node layer
    # b = 1024 FC hidden node layers
    y = tf.maximum((tf.matmul(reshape, wfc) + bfc), 0.01 * tf.matmul(reshape, wfc) + bfc) #tf.nn.relu(tf.matmul(reshape, w3) + b3)
    y = tf.nn.dropout(y, drop_rate)  # Dropout
    

    return y #tf.matmul(hidden, layer4_weights) + layer4_biases

  # Training computation.
  # w4 = 1024 batch sample * 6 classifier
  # b4 = 6 classifier
  # 1024 batch sample x 6 classifier
  logits =  tf.matmul(model(tf_train_dataset), w0fc) + b0fc
    
  # w4b = 1024 batch sample * 11 classifier
  # b4b = 11 classifier
  # 1024 batch sample x 11 classifier
  logits1 = tf.matmul(model(tf_train_dataset), w1fc) + b1fc
  logits2 = tf.matmul(model(tf_train_dataset), w2fc) + b2fc
  logits3 = tf.matmul(model(tf_train_dataset), w3fc) + b3fc
  logits4 = tf.matmul(model(tf_train_dataset), w4fc) + b4fc
  logits5 = tf.matmul(model(tf_train_dataset), w5fc) + b5fc
    
  loss0 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels0, logits=logits)
  loss1 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels1, logits=logits1)
  loss2 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels2, logits=logits2)
  loss3 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels3, logits=logits3)
  loss4 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels4, logits=logits4)
  loss5 = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels5, logits=logits5)
 # loss =  tf.reduce_mean(loss0 + loss1 + loss2 + loss3 + loss4 + loss5)

  loss= tf.reduce_mean(loss0) + tf.reduce_mean(loss1) + tf.reduce_mean(loss2) + tf.reduce_mean(loss3) + tf.reduce_mean(loss4) + tf.reduce_mean(loss5)
    
 # Optimizer.
 # optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  global_step = tf.Variable(0) #, trainable=False)  # count the number of steps taken.
 # learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
 # learning_rate = tf.train.exponential_decay(0.1, global_step, 200, 0.7) #, staircase=True)
 # optimizer = tf.train.GradientDescentOptimizer(1e-6).minimize(loss) #, global_step=global_step)
  optimizer =tf.train.AdamOptimizer(0.001).minimize(loss) #, global_step=global_step) 


  # Validation computation.
  logits_valid =  tf.matmul(model(tf_valid_dataset), w0fc) + b0fc
    
  # w4b = 1024 batch sample * 11 classifier
  # b4b = 11 classifier
  # 256 batch sample x 11 classifier
  logits1_valid = tf.matmul(model(tf_valid_dataset), w1fc) + b1fc
  logits2_valid = tf.matmul(model(tf_valid_dataset), w2fc) + b2fc
  logits3_valid = tf.matmul(model(tf_valid_dataset), w3fc) + b3fc
  logits4_valid = tf.matmul(model(tf_valid_dataset), w4fc) + b4fc
  logits5_valid = tf.matmul(model(tf_valid_dataset), w5fc) + b5fc

  # Predictions for the training, validation, and test data.
 # digit 1-5 prediction
  train_prediction1 = tf.nn.softmax(logits1)
  valid_prediction1 = tf.nn.softmax(logits1_valid)
  train_prediction2 = tf.nn.softmax(logits2)
  valid_prediction2 = tf.nn.softmax(logits2_valid)
  train_prediction3 = tf.nn.softmax(logits3)
  valid_prediction3 = tf.nn.softmax(logits3_valid)
  train_prediction4 = tf.nn.softmax(logits4)
  valid_prediction4 = tf.nn.softmax(logits4_valid)
  train_prediction5 = tf.nn.softmax(logits5)
  valid_prediction5 = tf.nn.softmax(logits5_valid)
  #Length of sequence prediction
  train_prediction0 = tf.nn.softmax(logits)
  valid_prediction0 = tf.nn.softmax(logits_valid)
    
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))


### Run graph session

In [ ]:
import time

t1 = time.time()

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

#def eval_accuracy(eval_l_preds, eval_preds, l_labels, labels, masks):
#    concatted = np.concatenate((np.reshape((eval_l_preds == l_labels), [-1, 1]), 
#                                (eval_preds * masks) == labels), axis=1)
#    return 100.0 * (np.sum([np.all(row) for row in concatted])) / len(labels)
#
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run() # sess.run(tf.global_variables_initializer())
  print('Initialized')
    
  for step in range(num_steps):
    offset = (step * batch_size) % (np.asarray(y1_train).shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :] #, :, :]
    
    #normalize data to mean 0 and unit variance
    batch_data = (batch_data - np.mean(batch_data))/np.std(batch_data)
    #batch_data = np.reshape(batch_data,(batch_size,image_size*image_size))
   # print(batch_data.shape)

  #  batch_data_valid = valid_dataset[offset:(offset + batch_size), :] #, :, :]
    
    batch_labels0 =  np.asarray(train_label0)[offset:(offset + batch_size), :]
    batch_labels1 =  np.asarray(train_label1)[offset:(offset + batch_size), :]
    batch_labels2 =  np.asarray(train_label2)[offset:(offset + batch_size), :]
    batch_labels3 =  np.asarray(train_label3)[offset:(offset + batch_size), :]
    batch_labels4 =  np.asarray(train_label4)[offset:(offset + batch_size), :]
    batch_labels5 =  np.asarray(train_label5)[offset:(offset + batch_size), :]    
   # print(batch_labels1.shape)
    feed_dict = {tf_train_dataset : batch_data
              #  , tf_valid_dataset : batch_data_valid
                , tf_train_labels0 : batch_labels0
                , tf_train_labels1 : batch_labels1
                , tf_train_labels2 : batch_labels2
                , tf_train_labels3 : batch_labels3
                , tf_train_labels4 : batch_labels4
                , tf_train_labels5 : batch_labels5
                }
    _, l, pred_train0, pred_train1, pred_train2, pred_train3, pred_train4, pred_train5  \
        , pred_valid0, pred_valid1, pred_valid2, pred_valid3, pred_valid4, pred_valid5  \
        = session.run([optimizer, loss, train_prediction0, train_prediction1, train_prediction2  \
                       , train_prediction3, train_prediction4, train_prediction5, valid_prediction0  \
                       , valid_prediction1, valid_prediction2, valid_prediction3 , valid_prediction4, valid_prediction5]  \
                      , feed_dict=feed_dict)
     
    #normalize data to mean 0 and unit variance
    batch_data = (batch_data - np.mean(batch_data))/np.std(batch_data)
    #batch_data = np.reshape(batch_data,(batch_size,image_size*image_size))
   # print(batch_data.shape)
   # batch_valid_labels0 =  np.asarray(valid_label0)[offset:(offset + batch_size), :]
   # batch_valid_labels1 =  np.asarray(valid_label1)[offset:(offset + batch_size), :]
   # batch_valid_labels2 =  np.asarray(valid_label2)[offset:(offset + batch_size), :]
   # batch_valid_labels3 =  np.asarray(valid_label3)[offset:(offset + batch_size), :]
   # batch_valid_labels4 =  np.asarray(valid_label4)[offset:(offset + batch_size), :]
   # batch_valid_labels5 =  np.asarray(valid_label5)[offset:(offset + batch_size), :]  
    
    if (step % 50 == 0):
      print('Minibatch training loss at step %d: %f' % (step, l))
      print("Training accuracy - digit count: {:.1f}%; digits 1-5: {:.1f}%, {:.1f}%, {:.1f}%, {:.1f}%, {:.1f}%".format( \
           accuracy(pred_train0, batch_labels0),accuracy(pred_valid1, batch_labels1)  \
        ,accuracy(pred_train2, batch_labels2), accuracy(pred_train3, batch_labels3)  \
        ,accuracy(pred_train4, batch_labels4), accuracy(pred_train5, batch_labels5)))
      avg_acc_train = (accuracy(pred_train0, batch_labels0) + accuracy(pred_train1, batch_labels1)
        + accuracy(pred_train2, batch_labels2) + accuracy(pred_train3, batch_labels3)
        + accuracy(pred_train4, batch_labels4) + accuracy(pred_train5, batch_labels5))/6
      print('Training avg accuracy: %.1f%%' % avg_acc_train)
        
      print("Validation accuracy - digit count: {:.1f}%; digits 1-5: {:.1f}%, {:.1f}%, {:.1f}%, {:.1f}%, {:.1f}%".format( \
           accuracy(pred_valid0, valid_label0[:batch_size]),accuracy(pred_valid1, valid_label1[:batch_size])  \
        ,accuracy(pred_valid2, valid_label2[:batch_size]), accuracy(pred_valid3, valid_label3[:batch_size])  \
        ,accuracy(pred_valid4, valid_label4[:batch_size]), accuracy(pred_valid5, valid_label5[:batch_size])))
      avg_acc_valid = (accuracy(pred_valid0, valid_label0[:batch_size]) + accuracy(pred_valid1, valid_label1[:batch_size])
        + accuracy(pred_valid2, valid_label2[:batch_size]) + accuracy(pred_valid3, valid_label3[:batch_size])
        + accuracy(pred_valid4, valid_label4[:batch_size]) + accuracy(pred_valid5, valid_label5[:batch_size]))/6
      print('Validation avg accuracy: %.1f%%' % avg_acc_train)
     # print("w1 mean: {} std: {}".format(np.mean(w), np.std(w)))
     # print("wy1 mean: {} std: {}".format(np.mean(w1), np.std(w1)))
      #print("label: ", batch_labels1)
    #  print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), L_valid))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

t2 = time.time()

print("Time: %0.2fs" % (t2 - t1))

Initialized
Minibatch training loss at step 0: 13.936771
Training accuracy - digit count: 27.7%; digits 1-5: 27.3%, 8.6%, 3.9%, 44.9%, 0.0%
Training avg accuracy: 18.9%
Validation accuracy - digit count: 19.9%; digits 1-5: 25.4%, 8.2%, 7.4%, 2.0%, 0.0%
Validation avg accuracy: 18.9%
Minibatch training loss at step 50: 6.926579
Training accuracy - digit count: 52.7%; digits 1-5: 24.6%, 23.8%, 75.4%, 94.9%, 100.0%
Training avg accuracy: 61.8%
Validation accuracy - digit count: 55.9%; digits 1-5: 30.9%, 23.8%, 75.8%, 96.1%, 100.0%
Validation avg accuracy: 61.8%
Minibatch training loss at step 100: 6.548984
Training accuracy - digit count: 55.9%; digits 1-5: 27.3%, 24.6%, 77.3%, 97.7%, 100.0%
Training avg accuracy: 63.7%
Validation accuracy - digit count: 47.7%; digits 1-5: 31.2%, 24.2%, 75.8%, 96.1%, 100.0%
Validation avg accuracy: 63.7%
Minibatch training loss at step 150: 6.491438
Training accuracy - digit count: 57.8%; digits 1-5: 23.8%, 26.6%, 70.7%, 96.9%, 100.0%
Training avg accurac